In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
import re

In [4]:


# Inicializa navegador
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

url = 'https://techguide.sh/pt-BR/path/java/'
driver.get(url)
time.sleep(3)  # Aguarda carregamento

# Suas expressões
expressoes_desejadas = []
pattern = re.compile(r'\*\*(.*?)\*\*')
with open('../nomes_botoes.txt', 'r') as arquivo:
    linhas = arquivo.readlines()
    for linha in linhas:
        
        match = pattern.search(linha)
        if match:
            expressoes_desejadas.append(match.group(1))

# Extrai HTML da página
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Localiza botões com <p> contendo as expressões

botoes_relevantes = []

for button in soup.find_all('button'):
    p_tag = button.find('p')
    if p_tag:
        texto = p_tag.get_text(strip=True)
        if any(expr.lower() in texto.lower() for expr in expressoes_desejadas):
            classes = button.get('class')
            if classes:
                botoes_relevantes.append({'texto': texto, 'classe': classes})  # Salva lista completa


# Lista para armazenar links
all_links = []

# Processa cada botão relevante
for botao in botoes_relevantes:
    try:
        condicoes_xpath = ' and '.join([f'contains(@class, "{c}")' for c in botao['classe']])
        xpath_botao = f'//button[{condicoes_xpath}]'

        button_element = driver.find_element(By.XPATH, xpath_botao)
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button_element)
        time.sleep(0.5)
        button_element.click()
        time.sleep(1)

        # Pega divs visíveis e captura links
        divs = driver.find_elements(By.TAG_NAME, 'div')
        visible_divs = [d for d in divs if d.is_displayed()]

        for div in visible_divs:
            links = div.find_elements(By.TAG_NAME, 'a')
            for link in links:
                href = link.get_attribute('href')
                text = link.text()
                category = botao['texto']
                if href and href not in all_links:
                    temp_dict = {}
                    temp_dict['categoria'] = category
                    temp_dict['texto'] = text
                    temp_dict['link'] = href 
                    all_links.append(temp_dict)


        # Fecha a div
        close_icon = driver.find_element(By.XPATH, '/html/body/div/div/div/div[2]/div[1]/div[2]')
        close_icon.click()
        time.sleep(1)

    except Exception as e:
        print(f'Erro no botão "{botao["texto"]}": {e}')
        continue

print(f'Total de links capturados: {len(all_links)}')
print(all_links)

driver.quit()


Erro no botão "Git e GitHub - Fundamentos": 'str' object is not callable
Erro no botão "HTTP - Fundamentos": Message: element click intercepted: Element <button class="sc-eedb5a62-0 jrfHUw Rjdddnaj6">...</button> is not clickable at point (292, 315). Other element would receive the click: <div class="sc-eedb5a62-0 bjXYHt modal-base">...</div>
  (Session info: chrome=138.0.7204.100); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementclickinterceptedexception
Stacktrace:
#0 0x5f523545a7da <unknown>
#1 0x5f5234eff2e0 <unknown>
#2 0x5f5234f57c6c <unknown>
#3 0x5f5234f55ad2 <unknown>
#4 0x5f5234f53172 <unknown>
#5 0x5f5234f52890 <unknown>
#6 0x5f5234f450ea <unknown>
#7 0x5f5234f768b2 <unknown>
#8 0x5f5234f44a6a <unknown>
#9 0x5f5234f76a7e <unknown>
#10 0x5f5234f9c71c <unknown>
#11 0x5f5234f76683 <unknown>
#12 0x5f5234f42b5b <unknown>
#13 0x5f5234f43f31 <unknown>
#14 0x5f523541f72b <unknown>
#15 0x5f5235423534 <unkno